1. metacritic url받아오기
2. /critic-reviews 랑 /user-reviews 내용 가져오기 => url + '/critic-reviews' 
3. 가져 온 텍스트 영어인지 필터
4. 텍스트로 파일 저장
==> wordcloud

In [1]:
import pandas as pd
import json

from langdetect import detect

import requests
from bs4 import BeautifulSoup

from wordcloud import WordCloud, STOPWORDS
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from textblob import TextBlob

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

import time

from datetime import datetime
from tqdm import tqdm

### 게임 리스트가 담긴 csv불러와서 검색에 용이하게 전처리!

In [2]:
df = pd.read_csv('../rank2023.csv')
df = df.loc[0:99,:]

game_list = []
for i in df.Game:
    game_name = i.replace(' ', '-').lower()
    game_name = game_name.replace("'", '')
    # 소문자, 특수기호 빼기
    game_list.append(game_name)
    
game_list

['pizza-tower',
 'resident-evil-4',
 'pineapple-on-pizza',
 'brotato',
 'the-murder-of-sonic-the-hedgehog',
 'dave-the-diver',
 'crab-champions',
 'hi-fi-rush',
 'volcano-princess',
 '东北之夏',
 'papas-freezeria-deluxe',
 'villagerhapsody',
 'dredge',
 'aimlabs',
 'steam-engine-simulator',
 'a-space-for-the-unbound',
 'bloody-hell',
 'octopath-traveler-ii',
 'handshakes',
 'hrot',
 'i-commissioned-some-bees-0',
 '候鸟',
 'boneraiser-minions',
 'your-only-move-is-hustle',
 'spin-rhythm-xd',
 'voidigo',
 'hogwarts-legacy',
 'the-outlast-trials',
 'halls-of-torment',
 'ghost-trick:-phantom-detective',
 'cassette-beasts',
 'your-turn-to-die--death-game-by-majority-',
 'barotrauma',
 'touhou\xa0hero\xa0of\xa0ice\xa0fairy',
 'smushi-come-home',
 'staffercase:-a-psychic-mystery-adventure',
 'amanda-the-adventurer',
 'the-test:-secrets-of-the-soul',
 'dr-livesey-rom-and-death-edition',
 'megs-monster',
 'endless-monday:-dreams-and-deadlines',
 'vertigo-2',
 'contraband-police',
 'bramble:-the-mount

### url이 스팀 웹에 없는 게임들이 있으므로 일일이 matacritic 웹사이트에서 검색

### 다수의 게임정보를 추출

## *class*

In [5]:
# 검색 결과 url을 만들어서 접속하기
def critic_reviews(game_name):
    url = 'https://www.metacritic.com/game/pc/' + game_name + '/critic-reviews'
    # setting driver
    return url

def user_reviews(game_name):
    url = 'https://www.metacritic.com/game/pc/' + game_name + '/user-reviews'
    return url

# critic reviews page에서 users reviews로 넘어가기
def move_to_users():
    driver.find_element(By.XPATH, '//*[@id="main"]/div[2]/ul/li[3]/span/span/a').click()

# 게시물 내용 가져오기
def get_content(driver):
    # 현재 페이지 html정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    parent_div = soup.select('div.review_body')
    # 본문 내용 가져오기
    text = []
    for n in range(len(parent_div)):
        if not parent_div[n].find(class_ = 'blurb blurb_expanded') == None:
            # review가 길어서 collapse되어 있다면, expanded된 텍스트를 뽑아와야한다.
            review = parent_div[n].find(class_ = 'blurb blurb_expanded').get_text()
            if detect(review) == 'en':
                text.append(review)
            else:
                pass          
        else:
            # expand 옵션이 없는 짧은 텍스트들은 그냥 리뷰를 받아오면 된다.
            txt = parent_div[n].get_text()
            if detect(txt) == 'en':
                text.append(txt)
            else:
                pass
    # 만들어진 text리스트를 str으로 변환
    textStr = ' '.join(map(str, text))
    
    # word cloud dictionary 데이터를 추출!
    stopwords = set(STOPWORDS)
    stopwords.add('game')
    stopwords.add('s')
    stopwords.add('re')
    stopwords.add('t')

    wc = WordCloud(stopwords=stopwords, margin = 10, random_state=1).generate(textStr)
    content = wc.words_
    
    return content


### 만들어진 CLASS와 게임리스트로 결과 값(results)을 추출

In [37]:
results = {}
for game_name in tqdm(game_list):
    try:
        reviews= {}
        path = '../driver/chromedriver.exe'
        driver = webdriver.Chrome(service = Service(path))
        
        #critics reviews
        driver.get(critic_reviews(game_name))
        data = get_content(driver)
        reviews['critics'] = data
        # user reviews
        move_to_users()
        data = get_content(driver)
        reviews['users'] = data   
        
        driver.close()
        results[game_name] = reviews
    except:
        pass

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [1:06:20<00:00, 39.80s/it]


### 결과값을 보고 추가할 stopwords가 있나 생각해보기

In [ ]:
total = []
for game_name in game_list:
    try:
        keys = results[game_name]['critics'].keys()
        key_list = list(keys)
        for i in key_list:
            total.append(i)
    except:
        pass

import nltk
import matplotlib.pyplot as plt
from nltk import FreqDist
from nltk.corpus import stopwords

# Download the stopwords corpus (if not already downloaded)
nltk.download('stopwords')

# Load the stopwords
stop_words = set(stopwords.words('english'))

# Assuming you have the text stored in a variable called "videogame_review"
# Replace "videogame_review" with your actual text data

# Tokenize the text
tokens = total #nltk.word_tokenize(text)

# Remove stopwords and non-alphabetic tokens
filtered_tokens = [token.lower() for token in tokens if token.lower() not in stop_words and token.isalpha()]

# Calculate the frequency distribution
fdist = FreqDist(filtered_tokens)

# Get the top 25 most frequent terms
top_terms = fdist.most_common(25)

# Print the top terms
for term, frequency in top_terms:
    print(term, frequency)

# Plot the frequency distribution
#fdist.plot(25, cumulative=False)
#plt.show()

## 결과값 json으로 저장!

In [44]:
# Specify the file path where you want to save the JSON data
file_path = r'C:\Users\Playdata\Desktop\data.json'

# Open the file in write mode and use json.dump() to write the dictionary data as JSON
with open(file_path, 'w') as file:
    json.dump(results, file)